ATIVIDADE 2

Baixa o seguinte arquivo CSV contendo dados sobre a população mundial de 2010-2020

https://www.kaggle.com/cityapiio/countries-population-2010-2020-data

Em seguida faça a importação do arquivo para o Colab ou Databricks e faça as atividades abaixo:

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.window import Window

**CRIANDO DATAFRAME**

In [0]:
df1 = (spark
      .read
      .format("csv")
      .option("header", "true")
      .option("inferschema", "true")
      .option("delimiter", ",")
      .load("/FileStore/tables/countries_general_info_historical_24_10_2021.csv")
)

In [0]:
df1.printSchema()

root
 |-- Name: string (nullable = true)
 |-- NativeName: string (nullable = true)
 |-- CallingCode: integer (nullable = true)
 |-- Iso3166P1Alpha2Code: string (nullable = true)
 |-- Iso3166P1Alpha3Code: string (nullable = true)
 |-- Iso3166P1NumericCode: integer (nullable = true)
 |-- Isni: string (nullable = true)
 |-- Population, 2010: integer (nullable = true)
 |-- Population, 2011: integer (nullable = true)
 |-- Population, 2012: integer (nullable = true)
 |-- Population, 2013: integer (nullable = true)
 |-- Population, 2014: integer (nullable = true)
 |-- Population, 2015: integer (nullable = true)
 |-- Population, 2016: integer (nullable = true)
 |-- Population, 2017: integer (nullable = true)
 |-- Population, 2018: integer (nullable = true)
 |-- Population, 2019: integer (nullable = true)



In [0]:
df1.display()

Name,NativeName,CallingCode,Iso3166P1Alpha2Code,Iso3166P1Alpha3Code,Iso3166P1NumericCode,Isni,"Population, 2010","Population, 2011","Population, 2012","Population, 2013","Population, 2014","Population, 2015","Population, 2016","Population, 2017","Population, 2018","Population, 2019"
Canada,Canada,1,CA,CAN,124,0000 0001 2238 5050,34004889,34339328,34714222,35082954,35437435,35702908,36109487,36545295,37065178,37593384
Japan,日本,81,JP,JPN,392,0000 0001 2364 9740,128070000,127833000,127629000,127445000,127276000,127141000,126994511,126785797,126529100,126264931
Norway,Kongeriket Norge,47,NO,NOR,578,0000 0001 2298 9524,4889252,4953088,5018573,5079623,5137232,5188607,5234519,5276968,5311916,5347896
Ireland,Eire,353,IE,IRL,372,0000 0001 2359 1198,4560155,4580084,4599533,4623816,4657740,4701957,4755335,4807388,4867316,4934040
Hungary,Magyarorszag,36,HU,HUN,348,0000 0001 2254 0380,10000023,9971727,9920362,9893082,9866468,9843028,9814023,9787966,9775564,9771141
Spain,Espana,34,ES,ESP,724,0000 0001 2324 3505,46576897,46742697,46773055,46620045,46480882,46444832,46484062,46593236,46797754,47133521
United States of America,United States of America,1,US,USA,840,0000 0001 2331 5230,309321666,311556874,313830990,315993715,318301008,320635163,322941311,324985539,326687501,328239523
Belgium,Koninkrijk Belgie,32,BE,BEL,56,null,10895586,11038264,11106932,11159407,11209057,11274196,11331422,11375158,11427054,11502704
Luxembourg,Letzebuerg,352,LU,LUX,442,0000 0001 2192 9204,506953,518347,530946,543360,556319,569604,582014,596336,607950,620001
Finland,Suomi,358,FI,FIN,246,null,5363352,5388272,5413971,5438972,5461512,5479531,5495303,5508214,5515525,5521606


1) Renomear alguma coluna

In [0]:
df2 = df1.select(F.col("Name").alias("Country"),
                 F.col("NativeName"),
                 F.col("CallingCode").alias("Zipcode"))
df2.show(5)

+-------+----------------+-------+
|Country|      NativeName|Zipcode|
+-------+----------------+-------+
| Canada|          Canada|      1|
|  Japan|            日本|     81|
| Norway|Kongeriket Norge|     47|
|Ireland|            Eire|    353|
|Hungary|    Magyarorszag|     36|
+-------+----------------+-------+
only showing top 5 rows



2) Mostrar os países que possuam população maior que 10 milhões de habitantes em 2012

In [0]:
df1.select(F.col("Name"), F.col("Population, 2012")).filter(F.col("Population, 2012") > 10000000).show()

+--------------------+----------------+
|                Name|Population, 2012|
+--------------------+----------------+
|              Canada|        34714222|
|               Japan|       127629000|
|               Spain|        46773055|
|United States of ...|       313830990|
|             Belgium|        11106932|
|              Poland|        38063164|
|               Italy|        59539717|
|              Greece|        11045011|
|              Turkey|        74653016|
|            Portugal|        10514844|
|               Egypt|        86422240|
|              Mexico|       117274155|
|               Kenya|        44343410|
|            Ethiopia|        92726971|
|               Ghana|        25996449|
|              France|        65659809|
|      United Kingdom|        63700215|
|People's Republic...|      1350695000|
|              Brazil|       199287296|
|              Russia|       143201721|
+--------------------+----------------+
only showing top 20 rows



3) Mostrar todos os dados apenas dos países que comecem com a letra B

In [0]:
df1.filter("Name like 'B%'").display()

Name,NativeName,CallingCode,Iso3166P1Alpha2Code,Iso3166P1Alpha3Code,Iso3166P1NumericCode,Isni,"Population, 2010","Population, 2011","Population, 2012","Population, 2013","Population, 2014","Population, 2015","Population, 2016","Population, 2017","Population, 2018","Population, 2019"
Belgium,Koninkrijk Belgie,32,BE,BEL,56,null,10895586,11038264,11106932,11159407,11209057,11274196,11331422,11375158,11427054,11502704
Brazil,Republica Federativa do Brasil,55,BR,BRA,76,0000 0001 2176 6679,195713635,197514534,199287296,201035903,202763735,204471769,206163058,207833831,209469333,211049527
Belarus,Рэспубліка Беларусь,375,BY,BLR,112,0000 0001 2171 2080,9490583,9473172,9464495,9465997,9474511,9489616,9501534,9498264,9483499,9417849
Bulgaria,България,359,BG,BGR,100,0000 0001 2325 4108,7395599,7348328,7305888,7265115,7223938,7177991,7127822,7075947,7025037,6975761
Bosnia and Herzegovina,Bosna i Hercegovina,387,BA,BIH,70,null,3705472,3661175,3604972,3542605,3482104,3429361,3386267,3351527,3323929,3301000
Belize,Belize,501,BZ,BLZ,84,0000 0001 2286 2013,322464,330237,338000,345715,353366,360933,368400,375769,383071,390353
Barbados,Barbados,1246,BB,BRB,52,0000 0001 0670 1869,282131,282987,283700,284296,284825,285324,285796,286233,286641,287025
Bahrain,البحرين,973,BH,BHR,48,0000 0004 0402 3058,1240860,1278151,1299943,1315029,1336075,1371851,1425791,1494074,1569439,1641172
Bolivia,Estado Plurinacional de Bolivia,591,BO,BOL,68,null,10048590,10212954,10377676,10542376,10706517,10869730,11031813,11192854,11353142,11513100
Bangladesh,গণপ্রজাতন্ত্রী বাংলাদেশ,880,BD,BGD,50,0000 0001 2188 6953,147575430,149273778,151007807,152764676,154520167,156256276,157970840,159670593,161356039,163046161


4) Criar um ranking de países por quantidade de habitantes (Do maior para o menor)

In [0]:
w0 = Window.orderBy(df1["Population, 2019"].desc())
df2 = df1.withColumn("rank", F.rank().over(w0))
df2.select(F.col("Name"), F.col("Population, 2019"), F.col("rank")).display()

Name,"Population, 2019",rank
People's Republic of China,1397715000,1
India,1366417754,2
United States of America,328239523,3
Indonesia,270625568,4
Pakistan,216565318,5
Brazil,211049527,6
Nigeria,200963599,7
Bangladesh,163046161,8
Russia,144406261,9
Mexico,127575529,10


5) Criar uma nova coluna com a seguinte informação: Caso o país tenha mais que 1 milhão de habitantes, colocar o texto “População com mais de 1 milhão de habitantes”. Caso contrário, colocar o texto: “População com menos de 1 milhão de habitantes”

In [0]:
df3 = (df1.withColumn("populacao_maior_menor", 
       F.when(F.col("Population, 2019") > 1000000, F.lit("População com mais de 1 milhão de habitantes"))
        .otherwise("População com menos de 1 milhão de habitantes")))

In [0]:
df3.select(F.col("Name"), F.col("Population, 2019"), F.col("populacao_maior_menor")).show()

+--------------------+----------------+---------------------+
|                Name|Population, 2019|populacao_maior_menor|
+--------------------+----------------+---------------------+
|              Canada|        37593384| População com mai...|
|               Japan|       126264931| População com mai...|
|              Norway|         5347896| População com mai...|
|             Ireland|         4934040| População com mai...|
|             Hungary|         9771141| População com mai...|
|               Spain|        47133521| População com mai...|
|United States of ...|       328239523| População com mai...|
|             Belgium|        11502704| População com mai...|
|          Luxembourg|          620001| População com men...|
|             Finland|         5521606| População com mai...|
|              Sweden|        10278887| População com mai...|
|              Poland|        37965475| População com mai...|
|           Lithuania|         2794137| População com mai...|
|       

6) Mostre a soma da população no ano de 2017.

In [0]:
df1.select(F.sum(F.col("Population, 2017"))).display()

"sum(Population, 2017)"
7448450667
